# Classification Tree

In [1]:
import pandas as pd
import numpy as np
import glob as glob
import json
import sklearn
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from random import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, explained_variance_score


import keras
from keras.layers import Dense, Dropout, RepeatVector, BatchNormalization, Convolution1D, Flatten, Lambda, Permute, MaxPooling1D, AlphaDropout
from keras.models import Sequential
from keras.utils import to_categorical
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.models import load_model

Using TensorFlow backend.


In [2]:
root_path = 'features_by_valence/'
label_set = ['low', 'high']
low_files = glob.glob(root_path + label_set[0] + '/*')
high_files = glob.glob(root_path + label_set[1] + '/*')

features = {'low':[], 'high':[]}

all_features = []

for filename in low_files:
    word_histogram = json.loads(open(filename).read())
    features['low'].append(word_histogram)
    all_features.append({'dictionary': word_histogram, 'label': 'low'})
for filename in high_files:
    word_histogram = json.loads(open(filename).read())
    features['high'].append(word_histogram)
    all_features.append({'dictionary': word_histogram, 'label': 'high'})

In [3]:
n_total = int(len(all_features))
n_train = int(n_total * 0.8)
n_test = int(n_total * 0.2)

print('total: ' + str(n_total))
print('train: ' + str(n_train))
print('test: ' + str(n_test))

total: 252
train: 201
test: 50


In [4]:
# one hot encoding

def word_list(percent_occurance):
    key_set = {}

    for feature_d in all_features:
        keys = feature_d['dictionary'].keys()
        for key in keys:
            key_set[key] = key_set.get(key, 0) + 1
    key_set
    # Return words that occur in the top percent_occurance% of documents
    key_set = dict([[key, key_set[key]] for key in key_set if (key_set[key]/float(n_total) > float(percent_occurance))])

    words_sorted_alphabetically = list(key_set.keys())
    words_sorted_alphabetically.sort()
    return words_sorted_alphabetically

In [5]:
def get_index(reference, key):
    try:
        return reference.index(key)
    except:
        return len(reference)

In [70]:
ohe = {}
i_2_label = {}

def one_hot_encode(reference, dictionary):
    ohe = np.zeros(len(reference) + 1)
    for key in dictionary.keys():
        pos = get_index(reference, key)
        ohe[pos] = dictionary[key]
    norm_lookup = float(np.linalg.norm(ohe))
    normalized_ohe = [float(item)/float(norm_lookup) for item in ohe]
    return normalized_ohe

In [7]:
def encode(y):
    if y == 'high':
        return 1
    else:
        return 0
    
def decode(y):
    if y == 1:
        return 'high'
    else:
        return 'low'

In [8]:
def validate(af):
    classifications = {'low':0, 'high':0}
    correct_preds = 0
    all_preds = 0
    for d_item in af:
        feature = d_item['ohe_feature']
        label = d_item['ohe_label']
        classification = clf.predict([feature])[0]
        classification = decode(classification)
        classifications[classification] = classifications[classification] + 1
        if classification == label:
            correct_preds = correct_preds + 1
        all_preds = all_preds + 1
    print(classifications)
    return float(correct_preds)/float(all_preds)

In [9]:
def preds_and_true(clf, af, ohe_words):
    correct_preds = 0
    all_preds = 0
    preds = []
    true = []
    for d_item in af:
        feature = d_item['ohe_feature']
        label = d_item['ohe_label']
        classification = clf.predict([feature])[0]
        preds.append(classification)
        true.append( label )
    return [preds, true]

In [10]:
def encode_features_and_words(all_features, word_threshold):
    ohe_words = word_list(word_threshold)
    
    for i, f in enumerate(all_features):
        ohe_feature = one_hot_encode(ohe_words, f['dictionary'])
        ohe_label = encode(f['label'])
        all_features[i]['ohe_feature'] = ohe_feature
        all_features[i]['ohe_label'] = ohe_label
    return [all_features, ohe_words]


In [71]:
def model(all_features, ohe_words):
    rounds = []
    for i in range(20):
        shuffle(all_features)
        
        train_features = all_features[0:n_train]
        test_features = all_features[n_train:n_total]
    
        X = [f['ohe_feature'] for f in train_features]
        y = [f['ohe_label'] for f in train_features]
        
        clf = GradientBoostingClassifier(random_state=0, learning_rate=0.005)
        clf.fit(X, y)
        
        preds, true = preds_and_true(clf, train_features, ohe_words)
        train_precision = precision_score(true, preds)
        train_acc = accuracy_score(true, preds)
        tr_ev = explained_variance_score(true,preds)
        
        preds, true = preds_and_true(clf, test_features, ohe_words)
        test_precision = precision_score(true, preds)
        test_acc = accuracy_score(true, preds)
        te_ev = explained_variance_score(true, preds)
        
        rounds.append({
            'train_precision': train_precision, 
            'train_acc': train_acc, 
            'tr_ev': tr_ev,
            'test_precision': test_precision,
            'test_acc': test_acc, 
            'te_ev': te_ev
            
        })
    totals = {}
    output = {}
    for ro in rounds: 
        for k in ro:
            ro_total_key = 'total-' + k
            totals[ro_total_key] = totals.get(ro_total_key, 0) + ro[k]
    for k in totals:
        ou_av_key = 'av-' + k
        output[ou_av_key] = (totals[k]/float(len(rounds)))
    return output
        

In [72]:
n = 0.25
all_features, ohe_words = encode_features_and_words(all_features, n)
model(all_features, ohe_words)

{'av-total-te_ev': -0.64557735201626365,
 'av-total-test_acc': 0.53921568627450966,
 'av-total-test_precision': 0.38383470695970695,
 'av-total-tr_ev': 0.60164987608211518,
 'av-total-train_acc': 0.88781094527363202,
 'av-total-train_precision': 0.99683335480454038}

In [13]:
# clf = GradientBoostingClassifier(random_state=0, learning_rate=0.005)
# clf.fit(X, y)

preds, true = preds_and_true(clf, train_features, ohe_words)
train_precision = precision_score(true, preds)
train_acc = accuracy_score(true, preds)
tr_ev = explained_variance_score(true,preds)

preds, true = preds_and_true(clf, test_features, ohe_words)
test_precision = precision_score(true, preds)
test_acc = accuracy_score(true, preds)
te_ev = explained_variance_score(true, preds)
print(str(n) + '.)' + "\n")
print({'train_precision': train_precision, 
    'train_acc': train_acc, 
    'tr_ev': tr_ev,
    'test_precision': test_precision,
    'test_acc': test_acc, 
    'te_ev': te_ev})

NameError: name 'X' is not defined

In [73]:
# n = 0.001
len(word_list(0.25))

2822

# Let's see if we can use deep learning to do the same thing

In [74]:
def get_train_test(all_features, n=0.25):
    all_features, ohe_words = encode_features_and_words(all_features, n)

    shuffle(all_features)

    train_features = all_features[0:n_train]
    test_features = all_features[n_train:n_total]

    X = [f['ohe_feature'] for f in train_features]
    y = [f['ohe_label'] for f in train_features]
    
    x_val = [f['ohe_feature'] for f in test_features]
    y_val = [f['ohe_label'] for f in test_features]
    return [X, y, x_val, y_val]

In [50]:
# 0.25 is a solid value for n. Lower values mean letting in larger amounts of words (features)
# Once validation accuracy is 66.67%, save the weights. That's the max. 
n = 0.25
X, y, x_val, y_val = get_train_test(all_features, n)

In [75]:
shape = len(word_list(n)) + 1
shape

2823

In [106]:
shape = len(word_list(n)) + 1

m1 = Sequential([
    BatchNormalization(input_shape=(shape,)),
    Dense(40, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(10, activation='relu'),
    Dropout(0.5),
    BatchNormalization(center=True),
    Dense(1, activation='sigmoid')   
])

m1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

sched = [[0.0001, 2], [0.001, 20], [0.01, 2], [0.1, 2], [0.5, 1], [0.1, 5], [0.01, 20], [0.001, 40], [0.0001, 80], [0.00005, 120]]


In [107]:
for info in sched:
    lr, epochs = info
    m1.optimizer.lr = lr
    m1.fit(np.array(X), np.array(y), epochs=epochs,  batch_size=64, validation_data=(np.array(x_val), np.array(y_val)))

Train on 201 samples, validate on 51 samples
Epoch 1/2
201/201 [==============================] - 5s 23ms/step - loss: 0.8447 - acc: 0.5124 - val_loss: 3.3356 - val_acc: 0.6667
Epoch 2/2
201/201 [==============================] - 0s 365us/step - loss: 0.8843 - acc: 0.4826 - val_loss: 2.6862 - val_acc: 0.6667
Train on 201 samples, validate on 51 samples
Epoch 1/20
201/201 [==============================] - 0s 419us/step - loss: 0.7690 - acc: 0.5174 - val_loss: 2.3379 - val_acc: 0.6667
Epoch 2/20
201/201 [==============================] - 0s 324us/step - loss: 0.8013 - acc: 0.5075 - val_loss: 2.0278 - val_acc: 0.6667
Epoch 3/20
201/201 [==============================] - 0s 363us/step - loss: 0.7829 - acc: 0.5174 - val_loss: 1.8375 - val_acc: 0.6667
Epoch 4/20
201/201 [==============================] - 0s 333us/step - loss: 0.7369 - acc: 0.5522 - val_loss: 1.6862 - val_acc: 0.6667
Epoch 5/20
201/201 [==============================] - 0s 351us/step - loss: 0.7172 - acc: 0.5622 - val_loss: 

201/201 [==============================] - 0s 374us/step - loss: 0.5591 - acc: 0.7015 - val_loss: 0.6840 - val_acc: 0.4706
Epoch 8/40
201/201 [==============================] - 0s 466us/step - loss: 0.6205 - acc: 0.6169 - val_loss: 0.6864 - val_acc: 0.4706
Epoch 9/40
201/201 [==============================] - 0s 448us/step - loss: 0.5923 - acc: 0.6716 - val_loss: 0.6862 - val_acc: 0.4510
Epoch 10/40
201/201 [==============================] - 0s 520us/step - loss: 0.6125 - acc: 0.6418 - val_loss: 0.6844 - val_acc: 0.4510
Epoch 11/40
201/201 [==============================] - 0s 533us/step - loss: 0.6212 - acc: 0.6219 - val_loss: 0.6839 - val_acc: 0.4706
Epoch 12/40
201/201 [==============================] - 0s 463us/step - loss: 0.6353 - acc: 0.6318 - val_loss: 0.6827 - val_acc: 0.4706
Epoch 13/40
201/201 [==============================] - 0s 522us/step - loss: 0.6063 - acc: 0.6866 - val_loss: 0.6822 - val_acc: 0.4706
Epoch 14/40
201/201 [==============================] - 0s 491us/step 

201/201 [==============================] - 0s 407us/step - loss: 0.5708 - acc: 0.6866 - val_loss: 0.6704 - val_acc: 0.5294
Epoch 28/80
201/201 [==============================] - 0s 335us/step - loss: 0.5611 - acc: 0.6816 - val_loss: 0.6693 - val_acc: 0.5294
Epoch 29/80
201/201 [==============================] - 0s 399us/step - loss: 0.5486 - acc: 0.7065 - val_loss: 0.6693 - val_acc: 0.5294
Epoch 30/80
201/201 [==============================] - 0s 413us/step - loss: 0.5375 - acc: 0.7214 - val_loss: 0.6704 - val_acc: 0.5294
Epoch 31/80
201/201 [==============================] - 0s 387us/step - loss: 0.5601 - acc: 0.6766 - val_loss: 0.6705 - val_acc: 0.5294
Epoch 32/80
201/201 [==============================] - 0s 398us/step - loss: 0.5102 - acc: 0.7562 - val_loss: 0.6709 - val_acc: 0.5098
Epoch 33/80
201/201 [==============================] - 0s 490us/step - loss: 0.5022 - acc: 0.7562 - val_loss: 0.6716 - val_acc: 0.5098
Epoch 34/80
201/201 [==============================] - 0s 425us/ste

201/201 [==============================] - 0s 373us/step - loss: 0.4989 - acc: 0.7264 - val_loss: 0.6736 - val_acc: 0.4902
Epoch 7/120
201/201 [==============================] - 0s 448us/step - loss: 0.5091 - acc: 0.7264 - val_loss: 0.6716 - val_acc: 0.5098
Epoch 8/120
201/201 [==============================] - 0s 513us/step - loss: 0.4823 - acc: 0.8109 - val_loss: 0.6709 - val_acc: 0.5098
Epoch 9/120
201/201 [==============================] - 0s 644us/step - loss: 0.5160 - acc: 0.7413 - val_loss: 0.6700 - val_acc: 0.5294
Epoch 10/120
201/201 [==============================] - 0s 411us/step - loss: 0.5320 - acc: 0.7662 - val_loss: 0.6704 - val_acc: 0.4902
Epoch 11/120
201/201 [==============================] - 0s 382us/step - loss: 0.4838 - acc: 0.7861 - val_loss: 0.6704 - val_acc: 0.4706
Epoch 12/120
201/201 [==============================] - 0s 549us/step - loss: 0.4992 - acc: 0.7463 - val_loss: 0.6703 - val_acc: 0.4706
Epoch 13/120
201/201 [==============================] - 0s 420us

201/201 [==============================] - 0s 379us/step - loss: 0.4373 - acc: 0.8308 - val_loss: 0.6695 - val_acc: 0.5294
Epoch 67/120
201/201 [==============================] - 0s 345us/step - loss: 0.4912 - acc: 0.7711 - val_loss: 0.6693 - val_acc: 0.5294
Epoch 68/120
201/201 [==============================] - 0s 365us/step - loss: 0.5006 - acc: 0.7512 - val_loss: 0.6680 - val_acc: 0.5294
Epoch 69/120
201/201 [==============================] - 0s 429us/step - loss: 0.4872 - acc: 0.7761 - val_loss: 0.6667 - val_acc: 0.5294
Epoch 70/120
201/201 [==============================] - 0s 329us/step - loss: 0.4996 - acc: 0.7612 - val_loss: 0.6668 - val_acc: 0.5294
Epoch 71/120
201/201 [==============================] - 0s 309us/step - loss: 0.4590 - acc: 0.8060 - val_loss: 0.6666 - val_acc: 0.5294
Epoch 72/120
201/201 [==============================] - 0s 409us/step - loss: 0.4761 - acc: 0.7960 - val_loss: 0.6664 - val_acc: 0.5294
Epoch 73/120
201/201 [==============================] - 0s 37